In [ ]:
-pimport pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
df_eurchf = pd.read_csv('/content/EUR-CHF Exchange Rate.csv')
#df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [ ]:
#df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_eurchf['Date'] = pd.to_datetime(df_eurchf.iloc[:, 0], infer_datetime_format=True)

In [ ]:
df_eurchf

,Date,value,log_difference
0,1999-01-04,1.6172,NaN
1,1999-01-05,1.6144,-0.001733
2,1999-01-06,1.6185,0.002536
3,1999-01-07,1.6134,-0.003156
4,1999-01-08,1.6134,0.000000
...,...,...,...
5983,2021-09-09,1.0838,-0.005245
5984,2021-09-10,1.0842,0.000369
5985,2021-09-11,1.0846,0.000369
5986,2021-09-13,1.0893,0.004324


In [ ]:
df_eurchf['EUR/CHF Log Difference'] = np.log(df_eurchf[' value']) - (np.log(df_eurchf[' value']).shift(1))
df_eurchf['CHF/EUR Log Difference'] = -1 * df_eurchf['EUR/CHF Log Difference']
df_eurchf = df_eurchf.dropna()

In [ ]:
df_eurchf

,Date,value,log_difference,EUR/CHF Log Difference,CHF/EUR Log Difference
1,1999-01-05,1.6144,-0.001733,-0.001733,0.001733
2,1999-01-06,1.6185,0.002536,0.002536,-0.002536
3,1999-01-07,1.6134,-0.003156,-0.003156,0.003156
4,1999-01-08,1.6134,0.000000,0.000000,-0.000000
5,1999-01-11,1.6098,-0.002234,-0.002234,0.002234
...,...,...,...,...,...
5983,2021-09-09,1.0838,-0.005245,-0.005245,0.005245
5984,2021-09-10,1.0842,0.000369,0.000369,-0.000369
5985,2021-09-11,1.0846,0.000369,0.000369,-0.000369
5986,2021-09-13,1.0893,0.004324,0.004324,-0.004324


In [ ]:
#ts_df = df_eurchf.merge(df_trends, how = 'inner', on = ['Date'])

In [ ]:
#ts_df = ts_df.drop(['log_difference',' value'], axis = 1)
#ts_df = ts_df.set_index(ts_df.iloc[:,0])
#ts_df = ts_df.drop(['Date', 'EUR/CHF Log Difference'], axis = 1)
ts_df = df_eurchf
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date', 'log_difference',' value', 'EUR/CHF Log Difference'], axis = 1)
ts_df

,CHF/EUR Log Difference
Date,
1999-01-05,0.001733
1999-01-06,-0.002536
1999-01-07,0.003156
1999-01-08,-0.000000
1999-01-11,0.002234
...,...
2021-09-09,0.005245
2021-09-10,-0.000369
2021-09-11,-0.000369


In [ ]:
#ts_df.iloc[:,1:155] = ts_df.iloc[:,1:155].diff()

In [ ]:
#ts_df = ts_df.iloc[1:,:]

In [ ]:
ts_df.head()

,CHF/EUR Log Difference
Date,
1999-01-05,0.001733
1999-01-06,-0.002536
1999-01-07,0.003156
1999-01-08,-0.000000
1999-01-11,0.002234


In [ ]:
# Creating a copy of the initial datagrame to make various transformations 
data = pd.DataFrame(ts_df.copy())

In [ ]:
window_size = 2
data['CHF/EUR_RealVol'] = data['CHF/EUR Log Difference'].rolling(window_size).std()*(252**0.5)

In [ ]:
data['CHF/EUR_RealVol'].quantile(0.75) #75th Percentile

0.04707111417477793

In [ ]:
# Adding the lag of the target variable from 6 steps back up to 24

for column in data:

  for i in range(1, 14):
      data[f"{column}_lag_{i}"] = data[column].shift(i)

In [ ]:
data['HighVolMoment_CHF/EUR'] = [1 if i > 0.05 else 0 for i in data['CHF/EUR_RealVol']]

In [ ]:
data

,CHF/EUR Log Difference,CHF/EUR_RealVol,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-05,0.001733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1999-01-06,-0.002536,0.047923,0.001733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1999-01-07,0.003156,0.063898,-0.002536,0.001733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1999-01-08,-0.000000,0.035426,0.003156,-0.002536,0.001733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063898,0.047923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1999-01-11,0.002234,0.025074,-0.000000,0.003156,-0.002536,0.001733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035426,0.063898,0.047923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,0.005245,0.064033,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,-0.002048,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,0.007286,1
2021-09-10,-0.000369,0.063023,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,1
2021-09-11,-0.000369,0.000002,-0.000369,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,0.063023,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0


In [ ]:
data = data.iloc[14:,:]

In [ ]:
data

,CHF/EUR Log Difference,CHF/EUR_RealVol,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-26,-0.002119,0.017484,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.003156,-0.002536,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0.063898,0.047923,0
1999-01-27,-0.003108,0.011104,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.003156,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0.063898,0
1999-01-28,0.000745,0.043257,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0
1999-01-29,0.001990,0.013971,0.000745,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,0.043257,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0
1999-02-01,-0.000062,0.023034,0.001990,0.000745,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.013971,0.043257,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,0.005245,0.064033,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,-0.002048,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,0.007286,1
2021-09-10,-0.000369,0.063023,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,1
2021-09-11,-0.000369,0.000002,-0.000369,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,0.063023,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0


In [ ]:
#fin_data = [data.iloc[:,0:1], data.iloc[:,156:]]
#final_data = pd.concat(fin_data, axis=1)

In [ ]:
final_data = data.copy()
final_data

,CHF/EUR Log Difference,CHF/EUR_RealVol,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-26,-0.002119,0.017484,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.003156,-0.002536,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0.063898,0.047923,0
1999-01-27,-0.003108,0.011104,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.003156,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0.063898,0
1999-01-28,0.000745,0.043257,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0
1999-01-29,0.001990,0.013971,0.000745,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,0.043257,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0
1999-02-01,-0.000062,0.023034,0.001990,0.000745,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.013971,0.043257,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,0.005245,0.064033,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,-0.002048,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,0.007286,1
2021-09-10,-0.000369,0.063023,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,1
2021-09-11,-0.000369,0.000002,-0.000369,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,0.063023,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0


#Classification

In [ ]:
# Creating a copy of the initial datagrame to make various transformations 
classification_df = pd.DataFrame(final_data.copy())

In [ ]:
classification_df

,CHF/EUR Log Difference,CHF/EUR_RealVol,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-26,-0.002119,0.017484,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.003156,-0.002536,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0.063898,0.047923,0
1999-01-27,-0.003108,0.011104,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.003156,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0.063898,0
1999-01-28,0.000745,0.043257,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,-0.000000,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0.035426,0
1999-01-29,0.001990,0.013971,0.000745,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.002234,0.043257,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0.025074,0
1999-02-01,-0.000062,0.023034,0.001990,0.000745,-0.003108,-0.002119,-0.000562,0.002494,0.000125,-0.005620,0.000626,-0.006152,0.002201,0.007387,0.003984,0.013971,0.043257,0.011104,0.017484,0.034303,0.026600,0.064483,0.070112,0.076082,0.093767,0.058203,0.038199,0.019641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,0.005245,0.064033,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,-0.002048,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,0.007286,1
2021-09-10,-0.000369,0.063023,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,-0.003527,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,1
2021-09-11,-0.000369,0.000002,-0.000369,0.005245,-0.000459,-0.002022,-0.001105,-0.000369,0.000737,-0.001474,-0.003233,0.001110,-0.006304,0.000186,0.003527,0.063023,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0


In [ ]:
y = classification_df['HighVolMoment_CHF/EUR']
X = classification_df.drop(['CHF/EUR Log Difference', 'CHF/EUR_RealVol', 'HighVolMoment_CHF/EUR'], axis=1)

#Train-Test Split 

In [ ]:
# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [ ]:
# reserve 30% of data for testing
X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

#Testing Models

##Benchmark

##Model 1: Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
LR1 = LogisticRegression(random_state=42, max_iter=10000).fit(X_train, y_train)

In [ ]:
prob_predictions1 = LR1.predict_proba(X_test)
pred1 = prob_predictions1[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="LR Predicted",
    mode="markers+lines", x=X_test.index, y=pred1
))

fig1.show()

In [ ]:
prediction1 = LR1.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction1)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction1)}")
print(f"Precision Score: {precision_score(y_test, prediction1)}")
print(f"F1 Score: {f1_score(y_test, prediction1)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred1, squared=False)}")

Recall Score: 0.028205128205128206
Accuracy Score: 0.7818080357142857
Precision Score: 0.4782608695652174
F1 Score: 0.053268765133171914
RMSE of Probability Score: 0.3991443798478938


##Model 2: SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
SGD2 = CalibratedClassifierCV(SGDClassifier(random_state=42, max_iter=10000)).fit(X_train, y_train)

In [ ]:
prob_predictions2 = SGD2.predict_proba(X_test)
pred2 = prob_predictions2[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SGD2 Predicted",
    mode="markers+lines", x=X_test.index, y=pred2
))

fig1.show()

In [ ]:
prediction2 = SGD2.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction2)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction2)}")
print(f"Precision Score: {precision_score(y_test, prediction2)}")
print(f"F1 Score: {f1_score(y_test, prediction2)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred2, squared=False)}")

Recall Score: 0.19743589743589743
Accuracy Score: 0.7952008928571429
Precision Score: 0.5877862595419847
F1 Score: 0.29558541266794625
RMSE of Probability Score: 0.3901554505188207


##Model 3: Passive Aggressive Classifier 

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
PAC3 = CalibratedClassifierCV(PassiveAggressiveClassifier(max_iter=10000, random_state=42)).fit(X_train, y_train)

In [ ]:
prob_predictions3 = PAC3.predict_proba(X_test)
pred3 = prob_predictions3[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="PAC3 Predicted",
    mode="markers+lines", x=X_test.index, y=pred3
))

fig1.show()

In [ ]:
prediction3 = PAC3.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction3)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction3)}")
print(f"Precision Score: {precision_score(y_test, prediction3)}")
print(f"F1 Score: {f1_score(y_test, prediction3)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred3, squared=False)}")

Recall Score: 0.18461538461538463
Accuracy Score: 0.7946428571428571
Precision Score: 0.5901639344262295
F1 Score: 0.28125
RMSE of Probability Score: 0.3885384788971612


##Model 4: Perceptron

In [ ]:
from sklearn.linear_model import Perceptron
Per4 = CalibratedClassifierCV(Perceptron(tol=1e-3, random_state=42)).fit(X_train, y_train)

In [ ]:
prob_predictions4 = Per4.predict_proba(X_test)
pred4 = prob_predictions4[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="Per4 Predicted",
    mode="markers+lines", x=X_test.index, y=pred4
))

fig1.show()

In [ ]:
prediction4 = Per4.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction4)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction4)}")
print(f"Precision Score: {precision_score(y_test, prediction4)}")
print(f"F1 Score: {f1_score(y_test, prediction4)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred4, squared=False)}")

Recall Score: 0.17435897435897435
Accuracy Score: 0.7957589285714286
Precision Score: 0.6071428571428571
F1 Score: 0.27091633466135456
RMSE of Probability Score: 0.3887620982303309


##Model 5: SVC

In [ ]:
from sklearn.svm import SVC
SVC5 = SVC(probability=True).fit(X_train, y_train)

In [ ]:
prob_predictions5 = SVC5.predict_proba(X_test)
pred5 = prob_predictions5[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVC5 Predicted",
    mode="markers+lines", x=X_test.index, y=pred5
))

fig1.show()

In [ ]:
prediction5 = SVC5.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction5)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction5)}")
print(f"Precision Score: {precision_score(y_test, prediction5)}")
print(f"F1 Score: {f1_score(y_test, prediction5)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred5, squared=False)}")

Recall Score: 0.07948717948717948
Accuracy Score: 0.7868303571428571
Precision Score: 0.5740740740740741
F1 Score: 0.1396396396396396
RMSE of Probability Score: 0.401402051324706


##Model 6: NuSVC

In [ ]:
from sklearn.svm import NuSVC
NuSVC6 = NuSVC(probability=True, nu=0.01).fit(X_train, y_train)

In [ ]:
prob_predictions6 = NuSVC6.predict_proba(X_test)
pred6 = prob_predictions6[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVC5 Predicted",
    mode="markers+lines", x=X_test.index, y=pred6
))

fig1.show()

In [ ]:
prediction6 = NuSVC6.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction6)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction6)}")
print(f"Precision Score: {precision_score(y_test, prediction6)}")
print(f"F1 Score: {f1_score(y_test, prediction6)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred6, squared=False)}")

Recall Score: 0.3487179487179487
Accuracy Score: 0.6813616071428571
Precision Score: 0.30022075055187636
F1 Score: 0.3226571767497034
RMSE of Probability Score: 0.4128007801191333


##Model 7: Linear SVC

In [ ]:
from sklearn.svm import LinearSVC
LinSVC7 = CalibratedClassifierCV(LinearSVC(random_state = 42)).fit(X_train, y_train)

In [ ]:
prob_predictions7 = LinSVC7.predict_proba(X_test)
pred7 = prob_predictions7[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="LinSVC7 Predicted",
    mode="markers+lines", x=X_test.index, y=pred7
))

fig1.show()

In [ ]:
prediction7 = LinSVC7.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction7)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction7)}")
print(f"Precision Score: {precision_score(y_test, prediction7)}")
print(f"F1 Score: {f1_score(y_test, prediction7)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred7, squared=False)}")

Recall Score: 0.18461538461538463
Accuracy Score: 0.7918526785714286
Precision Score: 0.5669291338582677
F1 Score: 0.2785299806576403
RMSE of Probability Score: 0.39083495651841443


##Model 8: K Neighbors

In [ ]:
from sklearn import neighbors
KNN8 = neighbors.KNeighborsClassifier().fit(X_train, y_train)

In [ ]:
prob_predictions8 = KNN8.predict_proba(X_test)
pred8 = prob_predictions8[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVR8 Predicted",
    mode="markers+lines", x=X_test.index, y=pred8
))

fig1.show()

In [ ]:
prediction8 = KNN8.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction8)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction8)}")
print(f"Precision Score: {precision_score(y_test, prediction8)}")
print(f"F1 Score: {f1_score(y_test, prediction8)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred8, squared=False)}")

Recall Score: 0.1564102564102564
Accuracy Score: 0.7751116071428571
Precision Score: 0.45185185185185184
F1 Score: 0.23238095238095238
RMSE of Probability Score: 0.42502625969294117


##Model 9: Gaussian Process Classification

In [ ]:
#from sklearn.gaussian_process.kernels import ConstantKernel, RBF
#from sklearn.gaussian_process import GaussianProcessClassifier

#kernel = ConstantKernel(constant_value=1.0, constant_value_bounds=(0.0, 10.0)) * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0)) + RBF(length_scale=2.0, length_scale_bounds=(0.0, 10.0))
#kernel = ConstantKernel() * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0))
#GPC9 = GaussianProcessClassifier(kernel=kernel, random_state=42).fit(X_train, y_train)

In [ ]:
#prob_predictions9 = GPC9.predict_proba(X_test)
#pred9 = prob_predictions9[:,1]

In [ ]:
#fig1 = go.Figure()

#fig1.add_trace(go.Scatter(
#    name="Raw Data",
#    mode="markers+lines", x=X_test.index, y=y_test
#))
#fig1.add_trace(go.Scatter(
#    name="GPC9 Predicted",
#    mode="markers+lines", x=X_test.index, y=pred9
#))

#fig1.show()

In [ ]:
#prediction9 = GPC9.predict(X_test)

In [ ]:
#Number Error Metrics

#print(f"Recall Score: {recall_score(y_test, prediction9)}")
#print(f"Accuracy Score: {accuracy_score(y_test, prediction9)}")
#print(f"Precision Score: {precision_score(y_test, prediction9)}")
#print(f"F1 Score: {f1_score(y_test, prediction9)}")

#Testing Probability Measure as EWS

#print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred9, squared=False)}")

#Model 10: Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB10 = GaussianNB().fit(X_train, y_train)

In [ ]:
prob_predictions10 = GNB10.predict_proba(X_test)
pred10 = prob_predictions10[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GPC9 Predicted",
    mode="markers+lines", x=X_test.index, y=pred10
))

fig1.show()

In [ ]:
prediction10 = GNB10.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction10)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction10)}")
print(f"Precision Score: {precision_score(y_test, prediction10)}")
print(f"F1 Score: {f1_score(y_test, prediction10)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred10, squared=False)}")

Recall Score: 0.1076923076923077
Accuracy Score: 0.78515625
Precision Score: 0.5316455696202531
F1 Score: 0.1791044776119403
RMSE of Probability Score: 0.45864657457461


#Model 11: Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTC11 = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
prob_predictions11 = DTC11.predict_proba(X_test)
pred11 = prob_predictions11[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="DTC11 Predicted",
    mode="markers+lines", x=X_test.index, y=pred11
))

fig1.show()

In [ ]:
prediction11 = DTC11.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction11)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction11)}")
print(f"Precision Score: {precision_score(y_test, prediction11)}")
print(f"F1 Score: {f1_score(y_test, prediction11)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred11, squared=False)}")

Recall Score: 0.4717948717948718
Accuracy Score: 0.7310267857142857
Precision Score: 0.4
F1 Score: 0.4329411764705882
RMSE of Probability Score: 0.518626276123486


##Model 12: Baggin KN Classifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
BAGGING12 = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5).fit(X_train, y_train)

In [ ]:
prob_predictions12 = BAGGING12.predict_proba(X_test)
pred12 = prob_predictions12[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="BAGGING12 Predicted",
    mode="markers+lines", x=X_test.index, y=pred12
))

fig1.show()

In [ ]:
prediction12 = BAGGING12.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction11)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction11)}")
print(f"Precision Score: {precision_score(y_test, prediction11)}")
print(f"F1 Score: {f1_score(y_test, prediction11)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred11, squared=False)}")

Recall Score: 0.4717948717948718
Accuracy Score: 0.7310267857142857
Precision Score: 0.4
F1 Score: 0.4329411764705882
RMSE of Probability Score: 0.518626276123486


##Model 13: Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF13 = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)

In [ ]:
prob_predictions13 = RF13.predict_proba(X_test)
pred13 = prob_predictions13[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="RF13 Predicted",
    mode="markers+lines", x=X_test.index, y=pred13
))

fig1.show()

In [ ]:
prediction13 = RF13.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction13)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction13)}")
print(f"Precision Score: {precision_score(y_test, prediction13)}")
print(f"F1 Score: {f1_score(y_test, prediction13)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred13, squared=False)}")

Recall Score: 0.4076923076923077
Accuracy Score: 0.83203125
Precision Score: 0.6943231441048034
F1 Score: 0.5137318255250405
RMSE of Probability Score: 0.3657557504308227


##Model 14: Extreme Randomized Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ETC14 = ExtraTreesClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
prob_predictions14 = ETC14.predict_proba(X_test)
pred14 = prob_predictions14[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="ETC14 Predicted",
    mode="markers+lines", x=X_test.index, y=pred14
))

fig1.show()

In [ ]:
prediction14 = ETC14.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction14)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction14)}")
print(f"Precision Score: {precision_score(y_test, prediction14)}")
print(f"F1 Score: {f1_score(y_test, prediction14)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred14, squared=False)}")

Recall Score: 0.23076923076923078
Accuracy Score: 0.8108258928571429
Precision Score: 0.6976744186046512
F1 Score: 0.3468208092485549
RMSE of Probability Score: 0.37649775302877214


#Model 15: AdaBoost CLassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
AB15 = AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)

In [ ]:
prob_predictions15 = AB15.predict_proba(X_test)
pred15 = prob_predictions15[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="AB15 Predicted",
    mode="markers+lines", x=X_test.index, y=pred15
))

fig1.show()

In [ ]:
prediction15 = AB15.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction15)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction15)}")
print(f"Precision Score: {precision_score(y_test, prediction15)}")
print(f"F1 Score: {f1_score(y_test, prediction15)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred15, squared=False)}")

Recall Score: 0.36666666666666664
Accuracy Score: 0.8125
Precision Score: 0.6163793103448276
F1 Score: 0.4598070739549839
RMSE of Probability Score: 0.49573019208689284


##Mode 16: Gradient Tree Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GTB16 = GradientBoostingClassifier(random_state = 42).fit(X_train, y_train)

In [ ]:
prob_predictions16 = GTB16.predict_proba(X_test)
pred16 = prob_predictions16[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GTB16 Predicted",
    mode="markers+lines", x=X_test.index, y=pred16
))

fig1.show()

In [ ]:
prediction16 = GTB16.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction16)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction16)}")
print(f"Precision Score: {precision_score(y_test, prediction16)}")
print(f"F1 Score: {f1_score(y_test, prediction16)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred16, squared=False)}")

Recall Score: 0.41025641025641024
Accuracy Score: 0.83203125
Precision Score: 0.6926406926406926
F1 Score: 0.5152979066022545
RMSE of Probability Score: 0.35765862411284577


##Model 17: Voting CLassifier

In [ ]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(random_state=1, max_iter=1000)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
  scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.80 (+/- 0.02) [Logistic Regression]
Accuracy: 0.81 (+/- 0.04) [Random Forest]
Accuracy: 0.81 (+/- 0.05) [naive Bayes]
Accuracy: 0.81 (+/- 0.04) [Ensemble]


##Model 18: XGBoost

In [ ]:
XGB18 = XGBClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
prob_predictions18 = XGB18.predict_proba(X_test)
pred18 = prob_predictions18[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="XGB18 Predicted",
    mode="markers+lines", x=X_test.index, y=pred18
))

fig1.show()

In [ ]:
prediction18 = XGB18.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction18)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction18)}")
print(f"Precision Score: {precision_score(y_test, prediction18)}")
print(f"F1 Score: {f1_score(y_test, prediction18)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred18, squared=False)}")

Recall Score: 0.4128205128205128
Accuracy Score: 0.8370535714285714
Precision Score: 0.71875
F1 Score: 0.5244299674267101
RMSE of Probability Score: 0.35771929778258926


#Model 19: MLP Classification

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP19 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [ ]:
prob_predictions19 = MLP19.predict_proba(X_test)
pred19 = prob_predictions19[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="MLP19 Predicted",
    mode="markers+lines", x=X_test.index, y=pred19
))

fig1.show()

In [ ]:
prediction19 = MLP19.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction19)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction19)}")
print(f"Precision Score: {precision_score(y_test, prediction19)}")
print(f"F1 Score: {f1_score(y_test, prediction19)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred19, squared=False)}")

Recall Score: 0.2512820512820513
Accuracy Score: 0.7940848214285714
Precision Score: 0.56
F1 Score: 0.3469026548672567
RMSE of Probability Score: 0.3925114163811415


##Model 20: MLPClassification Large

In [ ]:
MLP20 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [ ]:
prob_predictions20 = MLP20.predict_proba(X_test)
pred20 = prob_predictions20[:,1]

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="MLP20 Predicted",
    mode="markers+lines", x=X_test.index, y=pred20
))

fig1.show()

In [ ]:
prediction20 = MLP20.predict(X_test)

In [ ]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction20)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction20)}")
print(f"Precision Score: {precision_score(y_test, prediction20)}")
print(f"F1 Score: {f1_score(y_test, prediction20)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred20, squared=False)}")

Recall Score: 0.1717948717948718
Accuracy Score: 0.7879464285714286
Precision Score: 0.5403225806451613
F1 Score: 0.2607003891050583
RMSE of Probability Score: 0.39106188052878166
